## Creating meeting minutes from an audio file

I downloaded Denver city council meeting minute from this dataset.
The goal of this product is to use Audio to generate meeting minutes including actions

In [1]:
# !pip install datasets
!pip install pydub


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [1]:
from pydub import AudioSegment

pip install pydub--> installs the Python wrapper to work with audio, but needs ffmpeg underneath to actually work with .mp3 files

pip install --> for installing python packages/libraries
sudo apt install --> install system softwares

In [ ]:
!apt update
!apt install ffmpeg

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease                 
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease    
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
114 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  i965-va-driver intel-media-va-driver libaacs0 libaom0 libass9 libasyncns0
  libavc1394-0 libavcodec58 libavdevice58 libavfilter7 libavformat58
  libavresample4 libavutil56 libbdplus0 libbluray2 libbs2b0 libcaca0
  libcairo-gobj

In [3]:
# root_dir = "meeting/Audio&Transcripts/Denver/transcripts"


# if os.path.exists(root_dir):
#     print("path exists")
# else:
#     print("path doesnt exists")


path exists


In [40]:
!pwd

/home/zohreh/workspace/llm_engineering/week3


In [56]:
!pip install Ipython

import os
from pydub import AudioSegment
from IPython.display import Audio


# file_path = "/home/zohreh/workspace/llm_engineering/week3/meeting"
file_path = "./meeting"
mp3_file = os.path.join(file_path, "denver_0b39b5f6-dab6-4493-8769-0876474ff779.mp3" )

if os.path.exists(mp3_file):
    print("path exists")
else:
    print("path doesnt exists")


# audio = AudioSegment.from_file(mp3_file, format="mp3")
# first_25_min = audio[:25 * 60 * 1000]  # first 25 minutes
# first_25_min.export("output_audio/denver_25_min.mp3", format="mp3")
# from IPython.display import Audio
# audio_filename = "output_audio/denver_25_min.mp3"
# Audio(audio_filename)







[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
path exists


In [57]:
!pip install python-dotenv

import os 
import requests 
from IPython.display import Markdown, display, update_display
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
from dotenv import load_dotenv
from openai import OpenAI
!pip install ipywidgets


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [58]:
#constants 
audio_model = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [63]:
audio_filename = "output_audio/denver_25_min.mp3"
load_dotenv("2.env")
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
login(token=huggingface_token, add_to_git_credential=True)


In [65]:
openai_api_key = os.getenv("OPENAI_API_KEY")

openai = OpenAI(api_key=openai_api_key)

In [67]:
#use the whisper openai model to convert audio to text
audio_file = open(audio_filename, 'rb')
transcription = openai.audio.transcriptions.create(model = audio_model, file=audio_file, response_format='text')
print(transcription)

♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ Good evening, I want to welcome you to the Denver City Council meeting of Monday, May 18, 2015. Please rise and join Councilwoman Sussman in the Pledge of Allegiance. I pledge allegiance to the flag of the United States of America, and to the republic for which it stands, one nation, under God, indivisible, with liberty and justice for all. Thank you, Councilwoman. Madam Secretary, roll call. Brooks. Here. Brown. Here. Kenneche. Here. Layman. Here. Lopez. Here. Montero. Here. Nevitt. Here. Ortega. Here. Robb. Here. Shepard. Here. Sussman. Here. Mr. President. Here. 11 present. We have 11 members present. We have a quorum. Approval of minutes. Any corrections to the minutes of Monday, May 11, 2015? Seeing none, the minutes of May 11, 2015 stand approved. Council announcements. Any announcements from members of Council? Councilwoman Montero. Thank you, Mr. President. I just wanted to announce that last Thursday, on May 14th, we had a community

In [ ]:
## system and user prompt for llama to get summary from transcription from previous output from whisper-1 api model

In [75]:
system_message = "you are an assistant that produces minutes of meeting from transcript, with summary, key discussion point, takeaways and action items with owner in markdown"

user_prompt = f"below is an extract transcript of Denver council meeting. please write minutes in markdown, including summary with attendees, location and dates, discussion points for {transcription}"
messages = [
    {"role":"system", "content":system_message},
    {"role":"user", "content":user_prompt},

]



In [76]:
quant_config = BitsAndBytesConfig(
    load_in_4_bits=True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"

)

In [80]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config )
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer )
response = tokenizer.decode(outputs[0])   
print(response)

    


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:42<00:00, 10.66s/it]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

you are an assistant that produces minutes of meeting from transcript, with summary, key discussion point, takeaways and action items with owner in markdown<|eot_id|><|start_header_id|>user<|end_header_id|>

always.<|eot_id|><|start_header_id|>assistant<|end_header_id|>se write minutes in markdown, including summary with attendees, location and dates, discussion points for ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ ♪♪ Good evening, I want to welcome you to the Denver City Council meeting of Monday, May 18, 2015. Please rise and join Councilwoman Sussman in the Pledge of Allegiance. I pledge allegiance to the flag of the United States of America, and to the republic for which it stands, one nation, under God, indivisible, with liberty and justice for all. Thank you, Councilwoman. Madam Secretary, roll call. Brooks. Here. Brown. Here. Kenneche. Here. Layman. Here. Lo